## Advantages of Network Automation
In this notebook, we show the advantages of Network Automation when
comes to configure a bigger network. We extend the simple lab setup in
the previous section to include four more switches.
### 1. Multiple switches
#### 1.1 Create the topology
We add additional switches to the topology as in the figure belows.

![Alt text](images/02_multiple_switches_topology.png?raw=true \"Title\")

The connection from $S1$ to each switch $S2,\dots ,S5$ (for example the
link $S1-S2$, $S1-S3$, etc.) is out of band management network . $S1$ is
connected to the first gigabit port of each switch $S2,\dots ,S5$. In
this topology, assume that $S2,S3$ are core switches in the campus
network. They are connected to access switches $S4$ and $S5$.
#### 1.2. Initial Switch Configuration on $S2,\dots ,S5$
```bash
en
conf t
host S2
int vlan 1
 ip address 192.168.122.82 255.255.255.0
 no shut
enable password cisco
username doanh password cisco
line vty 0 4
 login local
 transport input all
end
copy run start
```
- Verify at the network automation container that it can telnet to
each switch.
```bash
telnet 192.168.122.82 # S2, username/password:doanh/cisco
telnet 192.168.122.83 # S3, username/password:doanh/cisco
telnet 192.168.122.84 # S4, username/password:doanh/cisco
telnet 192.168.122.85 # S5, username/password:doanh/cisco
```

#### 1.3. Create python scripts on the container
- Create the file myswitches contains the list of switches ip addresses.
- Copy the file Python3Switch.py to Python3Switch2.py and modify that
file.
```bash
cp Python3Switch.py Python3Switch2.py
nano Python3Switch2.py
```
- Modify the Python3Switch2.py as follow.

```python
import getpass
import telnetlib

f = open("myswitches")
user = input("Enter your telnet username: ")
password = getpass.getpass()
for IP in f:
    IP = IP.strip()
    print("Configuring switch "+str(IP))
    tn = telnetlib.Telnet(IP)

    tn.read_until(b"Username: ")
    tn.write(user.encode("ascii") + b"\n")
    if password:
        tn.read_until(b"Password: ")
        tn.write(password.encode("ascii") + b"\n")

    tn.write(b"enable\n")
    tn.write(b"cisco\n")
    tn.write(b"conf t\n")
    for n in range(2, 10):
        tn.write(b"vlan "+str(n).encode("ascii")+b"\n")
        tn.write(b"name Python_VLAN_"+str(n).encode("ascii")+b"\n")
    tn.write(b"end\n")
    tn.write(b"exit\n")

    print(tn.read_all().decode('ascii'))
```
- Verify the changes on each switch with `sh vlan brief`.

**Note:** Note that we should give privilege access for the user
with `username doanh privilege 15 password cisco`, so that we can go
directly to the EXEC mode when we telnet to the switch. With this 
change, the hard-code of password in the script could be avoid. The
couple of codes `tn.write(b"enable\n")` and `tn.write(b"cisco\n")`
from the above `Python3Switch2.py` file are no longer required.